# AHAB app Functions

In this notebook I will be testing the different functions needed to build up the app

In [253]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process
import json
from collections import Counter

In [254]:
with open('../data/CLEAN/audio_df.json', 'r') as audio:
    audio_products = json.load(audio)

with open('../data/CLEAN/camera_df.json', 'r') as cameras:
    camera_products = json.load(cameras)

with open('../data/CLEAN/lens_df.json', 'r') as lenses:
    lens_products = json.load(lenses)

with open('../data/CLEAN/lights_df.json', 'r') as lights:
    light_products = json.load(lights)

with open('../data/CLEAN/rental_places.json', 'r') as places:
    rental_places = json.load(places)

In [255]:
audiodf = pd.DataFrame(audio_products)
camerasdf = pd.DataFrame(camera_products)
lensesdf = pd.DataFrame(lens_products)
lightsdf = pd.DataFrame(light_products)
rentaldf = pd.DataFrame(rental_places)

In [256]:
user_input_list = input('Feed me your needs: ').lower().split(',')

In [257]:
user_input_list

['arri alexa 35',
 'arri/zeiss masterprime 18mm t1.3',
 'zoom h6',
 'nanlite pavotube 15c']

In [223]:
def find_products_in_dataframes(products):

    product_dict = {}
    
    for category, df in zip(['Audio', 'Cameras', 'Lenses', 'Lights'], [audiodf, camerasdf, lensesdf, lightsdf]):
        
        found_products = []
        
        for product in products:
            # Use fuzzy matching to find similar products
            
            found = df[df['name'].apply(lambda x: fuzz.partial_ratio(x, product)) > 95]['rental_place_id'].tolist()
            
            found_products.extend(found)
        
        product_dict[category] = found_products
    
    return product_dict


In [258]:
find_products_in_dataframes(user_input_list)

{'Audio': [1, 3], 'Cameras': [1, 2, 3], 'Lenses': [2], 'Lights': [1]}

In [259]:
product_dict = {'Audio': [1, 3], 'Cameras': [1, 2, 3], 'Lenses': [2], 'Lights': [1]}

In [248]:
# Function to find the most common rental place ID
def find_most_common_place_id(found_products_dict):

    all_ids = [item for sublist in found_products_dict.values() for item in sublist]
    count_ids = Counter(all_ids)
    most_common_id = count_ids.most_common(1)[0][0]
    
    return str(most_common_id)

In [260]:
find_most_common_place_id(product_dict)

'1'

In [250]:
# Function to get rental place info based on the most common ID
def get_rental_place_info(most_common_id):
    
    return rentaldf[rentaldf['rental_place_id'] == most_common_id]

In [261]:
get_rental_place_info('1')

,rental_place_id,rental_place_name,address,latitude,longitude,email,phone,website
0,1,WELAB,"C. de Albasanz, 59, 28037, Madrid",40.43856,-3.62394,info@welab.es,0034 911823355,https://www.welabplus.com


In [264]:
class AHABFinder:
    def __init__(self, audiodf, camerasdf, lensesdf, lightsdf, rental_places):
        
        self.audiodf = audiodf
        self.camerasdf = camerasdf
        self.lensesdf = lensesdf
        self.lightsdf = lightsdf
        self.rental_places = rental_places

    def find_products_in_dataframes(self, products):
        
        product_dict = {}
        
        for category, df in zip(['Audio', 'Cameras', 'Lenses', 'Lights'], [self.audiodf, self.camerasdf, self.lensesdf, self.lightsdf]):
            found_products = []
            for product in products:
                found = df[df['name'].apply(lambda x: fuzz.partial_ratio(x, product)) > 95]['rental_place_id'].tolist()
                found_products.extend(found)
            
            product_dict[category] = found_products
        
        return product_dict

    def find_most_common_place_id(self, found_products_dict):
        
        all_ids = [item for sublist in found_products_dict.values() for item in sublist]
        count_ids = Counter(all_ids)
        most_common_id = count_ids.most_common(1)[0][0]
        
        return str(most_common_id)

    def get_rental_place_info(self, rental_place_id):
        
        return self.rental_places[self.rental_places['rental_place_id'] == rental_place_id]

    def find_rental_place_for_products(self):
        
        user_input = input("Enter a list of products separated by commas: ")
        input_products = [prod.strip() for prod in user_input.split(',')]
        
        found_products_dict = self.find_products_in_dataframes(input_products)
        most_common_id = self.find_most_common_place_id(found_products_dict)
        rental_place_info = self.get_rental_place_info(most_common_id)
        
        return rental_place_info

In [265]:
finder = AHABFinder(audiodf, camerasdf, lensesdf, lightsdf, rental_places)

result = finder.find_rental_place_for_products()

result

TypeError: list indices must be integers or slices, not str